In [13]:
%load_ext autoreload
%autoreload 2

import os
import polars as pl

from src.seqeuencing_data_preprocessing import preprocess_seq_data
from src.data_preprocessing import OmicDataSplitter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
mrna = pl.read_csv(
    "tcga_brca_data/raw/mrna.csv"
)
mrna

gene,TCGA-BH-A1ET-01,TCGA-AR-A1AI-01,TCGA-AN-A0AL-01,TCGA-BH-A0HK-01,TCGA-A8-A09G-01,TCGA-E2-A109-01,TCGA-AN-A0AM-01,TCGA-A2-A0CU-01,TCGA-AR-A1AY-01,TCGA-AN-A0FF-01,TCGA-BH-A0B5-01,TCGA-A8-A06Z-01,TCGA-C8-A12U-01,TCGA-C8-A131-01,TCGA-A2-A0YG-01,TCGA-A2-A0CV-01,TCGA-B6-A0RT-01,TCGA-A7-A0CG-01,TCGA-C8-A1HI-01,TCGA-BH-A18S-01,TCGA-A8-A07L-01,TCGA-BH-A0BA-01,TCGA-BH-A0H7-01,TCGA-BH-A0W5-01,TCGA-AO-A03T-01,TCGA-BH-A0BJ-01,TCGA-A2-A0EM-01,TCGA-AN-A0XN-01,TCGA-A8-A084-01,TCGA-AN-A0FV-01,TCGA-BH-A0DQ-01,TCGA-AR-A0TR-01,TCGA-AN-A0XW-01,TCGA-BH-A18G-01,TCGA-BH-A0AY-01,TCGA-A7-A0D9-01,…,TCGA-AN-A0XU-01,TCGA-A2-A0EQ-01,TCGA-D8-A141-01,TCGA-A8-A07O-01,TCGA-B6-A0IO-01,TCGA-E2-A158-01,TCGA-A8-A07P-01,TCGA-A8-A099-01,TCGA-A2-A0YH-01,TCGA-AN-A04C-01,TCGA-BH-A0AZ-01,TCGA-AN-A0AK-01,TCGA-AO-A12E-01,TCGA-A2-A04N-01,TCGA-A2-A0D2-01,TCGA-A2-A0SW-01,TCGA-AR-A0U2-01,TCGA-BH-A0E2-01,TCGA-D8-A147-01,TCGA-D8-A13Z-01,TCGA-E2-A152-01,TCGA-A8-A06Y-01,TCGA-A2-A0T4-01,TCGA-BH-A0HY-01,TCGA-AQ-A04J-01,TCGA-BH-A1ES-01,TCGA-AO-A0J9-01,TCGA-BH-A0HO-01,TCGA-BH-A0B4-01,TCGA-A8-A09B-01,TCGA-AO-A0J2-01,TCGA-E2-A1BC-01,TCGA-A2-A04Q-01,TCGA-E2-A14O-01,TCGA-E2-A15F-01,TCGA-BH-A0DZ-01,TCGA-E2-A107-01
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ARHGEF10L""",9.471,11.2755,10.2441,9.3233,8.6366,9.8626,9.46,11.2247,9.4109,10.5067,10.2724,9.3159,10.8544,9.1495,8.8155,8.9437,10.7764,8.6821,9.2141,8.6442,9.5089,8.7218,9.4944,9.7738,9.9518,9.5628,9.3572,9.172,9.6494,9.0805,9.2398,9.9834,9.3135,11.0812,9.1019,10.9025,…,10.0025,10.2059,9.6739,9.6396,10.1613,9.2199,9.0376,8.3392,9.9363,9.19,9.4325,9.9483,9.8929,9.9831,10.1611,9.1269,9.4702,9.6913,10.4024,9.5647,9.8236,8.7745,9.1123,9.34,9.7674,9.7386,9.6978,8.6016,9.2534,10.0525,8.9836,9.0303,10.064,8.9483,10.0334,9.704,10.3434
"""HIF3A""",2.4075,5.553,2.3294,5.2633,3.1131,0.944,0.4316,2.9769,4.637,3.4019,2.5858,0.5913,0.8993,2.1272,0.0,3.0291,7.242,3.7612,2.8502,1.7164,0.4251,5.2743,1.2099,3.5127,3.8956,1.8117,1.4728,1.5814,1.8903,2.9125,5.0026,3.0731,1.1158,2.0444,3.9797,2.8431,…,6.9861,1.1658,1.8964,1.135,1.8607,4.3358,1.0262,0.0,2.8848,1.2742,3.4037,0.0,4.4619,1.7558,2.1132,3.1533,2.6489,0.4777,4.7636,2.1727,2.3097,0.4999,4.2523,0.0,8.7758,1.4002,5.0152,0.8863,3.0086,3.1148,1.2778,6.9578,2.2799,1.7676,0.8836,3.587,2.6529
"""RNF17""",0.0,0.0,0.0,0.0,0.0,0.6923,0.0,0.0,1.3043,0.0,0.0,0.3257,0.8993,0.0,0.0,0.0,0.71,0.0,0.0,0.0,0.4251,0.0,0.9891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.4272,0.0,0.0,0.9238,0.7294,0.0,0.0,0.0,0.0,0.438,0.0,0.0,0.0,0.0,0.0,1.2118,0.0,0.0,0.4173,1.2324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3593,0.0,0.0,0.3617,0.0,0.0,0.0,0.0,0.0,0.0
"""RNF10""",11.9416,11.3922,12.8216,11.6027,12.2614,11.9256,11.8298,11.5801,11.8127,12.2998,11.9089,11.8353,11.7444,11.7985,12.2153,12.0265,11.3035,11.5211,12.8131,11.6078,11.7502,12.3758,12.3283,11.7013,12.146,11.7872,11.6602,11.3834,12.4,12.09,12.2941,12.2632,11.7749,12.0761,11.8532,11.7959,…,11.1178,11.9862,11.9092,11.2711,12.5781,11.8904,11.8852,11.7441,11.8044,11.6267,11.9718,12.3737,12.0904,12.185,11.7749,12.0454,12.2348,11.9049,11.4056,12.1598,12.1153,12.601,11.721,11.4961,11.3629,12.2986,12.4817,11.9571,11.6441,12.023,11.2162,11.8271,11.7065,12.7112,12.0135,11.5199,12.4371
"""RNF11""",11.7528,11.332,11.1384,11.2566,11.8996,10.862,10.8835,11.116,11.1572,10.9935,10.7238,11.7705,11.6908,11.4236,11.3417,11.5781,10.652,11.5159,11.7781,11.4076,11.2024,11.4299,11.2033,10.8349,11.2574,11.3273,11.7259,11.5843,10.9288,10.8986,11.6315,11.3874,11.4345,10.9959,10.7975,11.164,…,11.1285,11.3253,11.0164,11.0634,11.0549,10.6569,11.9269,10.5351,10.5196,11.7137,11.0161,10.7525,11.301,11.3242,10.9105,11.5032,10.5823,11.0442,10.5655,11.5122,11.8669,10.4113,10.7345,11.5667,11.3039,11.5836,10.8825,11.3023,10.4376,10.9447,10.7171,

In [5]:
annotations = pl.read_csv("tcga_brca_data/raw/annotations.csv")
annotations

# annotations = annotations.with_columns(
#     (pl.col("PAM50_mRNA_nature2012").rank("dense") - 1).alias("classes")
# )
# # annotations
# annotations.filter(
#     pl.col("sampleID").is_in(mrna.columns)
# ).write_csv("tcga_brca_data/raw/annotations.csv")

sampleID,PAM50_mRNA_nature2012,classes
str,str,i64
"""TCGA-A1-A0SD-01""","""Luminal A""",2
"""TCGA-A1-A0SE-01""","""Luminal A""",2
"""TCGA-A1-A0SH-01""","""Luminal A""",2
"""TCGA-A1-A0SJ-01""","""Luminal A""",2
"""TCGA-A1-A0SK-01""","""Basal-like""",0
…,…,…
"""TCGA-E2-A1B4-01""","""Luminal A""",2
"""TCGA-E2-A1B5-01""","""Basal-like""",0
"""TCGA-E2-A1B6-01""","""Luminal A""",2


In [7]:
annotations['classes'].value_counts()

classes,count
i64,u32
3,122
4,8
2,219
1,55
0,85


In [8]:
annotations = annotations.filter(
    pl.col('PAM50_mRNA_nature2012') != "Normal-like"
)
nonnormal_samples = annotations['sampleID'].to_list()

mrna = mrna.select(['gene'] + nonnormal_samples)

In [9]:
annotations['classes'].value_counts()

classes,count
i64,u32
1,55
3,122
0,85
2,219


In [26]:
mrna

gene,TCGA-A1-A0SD-01,TCGA-A1-A0SE-01,TCGA-A1-A0SH-01,TCGA-A1-A0SJ-01,TCGA-A1-A0SK-01,TCGA-A1-A0SM-01,TCGA-A1-A0SO-01,TCGA-A2-A04N-01,TCGA-A2-A04P-01,TCGA-A2-A04Q-01,TCGA-A2-A04R-01,TCGA-A2-A04T-01,TCGA-A2-A04U-01,TCGA-A2-A04V-01,TCGA-A2-A04W-01,TCGA-A2-A04X-01,TCGA-A2-A04Y-01,TCGA-A2-A0CL-01,TCGA-A2-A0CM-01,TCGA-A2-A0CP-01,TCGA-A2-A0CQ-01,TCGA-A2-A0CS-01,TCGA-A2-A0CT-01,TCGA-A2-A0CU-01,TCGA-A2-A0CV-01,TCGA-A2-A0CW-01,TCGA-A2-A0D0-01,TCGA-A2-A0D1-01,TCGA-A2-A0D2-01,TCGA-A2-A0D3-01,TCGA-A2-A0D4-01,TCGA-A2-A0EM-01,TCGA-A2-A0EN-01,TCGA-A2-A0EO-01,TCGA-A2-A0EQ-01,TCGA-A2-A0ER-01,…,TCGA-E2-A14W-01,TCGA-E2-A14X-01,TCGA-E2-A14Y-01,TCGA-E2-A14Z-01,TCGA-E2-A150-01,TCGA-E2-A152-01,TCGA-E2-A153-01,TCGA-E2-A154-01,TCGA-E2-A155-01,TCGA-E2-A156-01,TCGA-E2-A158-01,TCGA-E2-A159-01,TCGA-E2-A15A-01,TCGA-E2-A15C-01,TCGA-E2-A15D-01,TCGA-E2-A15E-01,TCGA-E2-A15F-01,TCGA-E2-A15G-01,TCGA-E2-A15H-01,TCGA-E2-A15I-01,TCGA-E2-A15J-01,TCGA-E2-A15K-01,TCGA-E2-A15L-01,TCGA-E2-A15M-01,TCGA-E2-A15O-01,TCGA-E2-A15P-01,TCGA-E2-A15R-01,TCGA-E2-A15S-01,TCGA-E2-A15T-01,TCGA-E2-A1AZ-01,TCGA-E2-A1B0-01,TCGA-E2-A1B1-01,TCGA-E2-A1B4-01,TCGA-E2-A1B5-01,TCGA-E2-A1B6-01,TCGA-E2-A1BC-01,TCGA-E2-A1BD-01
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ARHGEF10L""",9.1657,9.7179,8.8669,9.7663,9.8632,8.8234,8.6701,9.9831,10.5102,10.064,9.5425,10.449,9.4763,10.5333,8.9774,8.9665,9.3774,9.5693,11.4132,9.6413,10.3112,9.2703,9.0876,11.2247,8.9437,9.741,8.3209,11.1395,10.1611,9.2467,10.9353,9.3572,10.0545,9.778,10.2059,10.0794,…,8.9611,10.7636,9.9899,9.8046,11.2344,9.8236,9.984,9.2405,9.8415,10.3731,9.2199,10.1219,9.1705,10.0851,9.2262,9.7065,10.0334,10.0949,9.4848,9.1406,9.2531,9.9412,9.3713,8.453,9.376,9.1228,9.1934,10.0346,9.8209,9.1036,8.5746,9.3223,10.4367,8.9027,8.6965,9.0303,10.7991
"""HIF3A""",2.4935,3.3983,1.6631,6.1765,7.497,1.8177,8.9699,1.7558,6.9166,2.2799,1.4911,9.2573,6.3214,2.9105,2.5516,1.6612,3.6294,3.6849,0.0,4.8982,0.4407,2.3655,1.1726,2.9769,3.0291,3.35,1.7954,1.9355,2.1132,0.0,4.0158,1.4728,4.2849,2.5901,1.1658,1.8923,…,1.7638,4.8435,2.8783,2.9498,2.9782,2.3097,1.6892,1.9935,2.1329,0.7806,4.3358,4.1235,0.7135,0.848,3.8032,4.4089,0.8836,2.4616,1.1422,3.0117,0.4366,1.3397,1.6929,4.9786,0.8672,1.7922,0.3056,5.0409,0.3936,5.7418,1.3134,4.919,1.8654,4.8873,4.6748,6.9578,2.1772
"""RNF17""",0.0,0.0,0.4449,0.0,0.7051,1.178,0.0,0.0,0.6028,0.0,0.0,0.0,0.0,2.0906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.7299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.8064,0.0,0.0,0.0,0.0,0.638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0372,0.0,0.0,1.4776,0.0,0.8114
"""RNF10""",11.7868,11.3219,11.8779,11.502,12.099,11.9934,11.6209,12.185,12.1741,11.7065,11.9362,11.3839,12.5415,12.22,12.0188,12.5742,12.0841,11.6492,12.0196,12.0527,12.2531,12.3055,11.1702,11.5801,12.0265,12.2589,11.8358,13.0469,11.7749,11.9473,12.1437,11.6602,12.0735,11.881,11.9862,11.965,…,12.3585,11.8525,11.7697,12.3889,11.3972,12.1153,11.7244,11.5464,11.9198,12.0953,11.8904,12.0105,12.2024,11.9198,11.9175,11.7502,12.0135,12.2843,12.3655,12.0196,11.9345,11.9704,11.88,11.7094,12.4611,12.4242,11.7385,12.4475,11.8324,11.416,11.7854,11.7005,12.428,12.6904,12.3403,11.8271,12.2783
"""RNF11""",11.0131,11.2617,11.8526,11.4405,9.9933,11.9332,11.0278,11.3242,11.2632,10.7574,10.4421,10.6897,10.9603,11.2698,11.9258,11.0061,10.9864,11.0977,11.1327,11.1117,11.1687,10.7255,11.4965,11.116,11.5781,10.6494,10.838,11.5712,10.9105,12.0452,10.1117,11.7259,10.9669,11.3562,11.3253,11.5178,…,11.3929,11.1539,11.0064,10.2678,11.758,11.8669,11.1408,10.3065,11.696,11.6226,10.6569,12.1943,10.6067,11.4468,11.066,11.5324,10.837,11.112,10.7207,11.3718,10.688,10.898,11.183,11.415,10.718,10.5873,10.1383,10.8017,11.1716,11.2437,10.9101,11.6

In [10]:
mrna = preprocess_seq_data(
    df = mrna,
    annotation_cols=['gene'],
    min_median_expression=2,
    min_detection_rate=0.4,
    top_n=10000,
    cpm_normalization=False,
    log_transform=False,
)

In [11]:
mrna

gene,TCGA-A1-A0SD-01,TCGA-A1-A0SE-01,TCGA-A1-A0SH-01,TCGA-A1-A0SJ-01,TCGA-A1-A0SK-01,TCGA-A1-A0SM-01,TCGA-A1-A0SO-01,TCGA-A2-A04N-01,TCGA-A2-A04P-01,TCGA-A2-A04Q-01,TCGA-A2-A04R-01,TCGA-A2-A04T-01,TCGA-A2-A04U-01,TCGA-A2-A04V-01,TCGA-A2-A04W-01,TCGA-A2-A04X-01,TCGA-A2-A04Y-01,TCGA-A2-A0CL-01,TCGA-A2-A0CM-01,TCGA-A2-A0CP-01,TCGA-A2-A0CQ-01,TCGA-A2-A0CS-01,TCGA-A2-A0CT-01,TCGA-A2-A0CU-01,TCGA-A2-A0CV-01,TCGA-A2-A0CW-01,TCGA-A2-A0D0-01,TCGA-A2-A0D1-01,TCGA-A2-A0D2-01,TCGA-A2-A0D3-01,TCGA-A2-A0D4-01,TCGA-A2-A0EM-01,TCGA-A2-A0EN-01,TCGA-A2-A0EO-01,TCGA-A2-A0EQ-01,TCGA-A2-A0ER-01,…,TCGA-E2-A14W-01,TCGA-E2-A14X-01,TCGA-E2-A14Y-01,TCGA-E2-A14Z-01,TCGA-E2-A150-01,TCGA-E2-A152-01,TCGA-E2-A153-01,TCGA-E2-A154-01,TCGA-E2-A155-01,TCGA-E2-A156-01,TCGA-E2-A158-01,TCGA-E2-A159-01,TCGA-E2-A15A-01,TCGA-E2-A15C-01,TCGA-E2-A15D-01,TCGA-E2-A15E-01,TCGA-E2-A15F-01,TCGA-E2-A15G-01,TCGA-E2-A15H-01,TCGA-E2-A15I-01,TCGA-E2-A15J-01,TCGA-E2-A15K-01,TCGA-E2-A15L-01,TCGA-E2-A15M-01,TCGA-E2-A15O-01,TCGA-E2-A15P-01,TCGA-E2-A15R-01,TCGA-E2-A15S-01,TCGA-E2-A15T-01,TCGA-E2-A1AZ-01,TCGA-E2-A1B0-01,TCGA-E2-A1B1-01,TCGA-E2-A1B4-01,TCGA-E2-A1B5-01,TCGA-E2-A1B6-01,TCGA-E2-A1BC-01,TCGA-E2-A1BD-01
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""CYP2A6""",0.7326,0.9907,0.0,7.0156,0.3953,0.0,0.0,0.4815,0.6028,0.4706,5.0547,0.4305,1.3554,2.6545,4.0591,0.7832,0.4164,2.9395,0.5474,1.2268,3.7932,8.0671,2.4851,2.8297,1.7191,2.8724,0.0,0.0,1.0765,0.0,1.0461,13.0051,3.158,1.6563,0.0,4.1354,…,1.5443,1.1688,0.0,11.6415,0.8555,0.0,0.0,3.0799,1.09,0.0,0.0,0.0,4.0318,1.1375,2.3794,3.771,8.3743,8.0539,11.3892,4.3017,4.3529,4.1877,0.0,14.5701,2.3563,8.4825,1.6428,2.9821,3.0383,0.0,1.5143,2.9513,0.6123,15.9084,0.0,2.3155,1.5912
"""CLEC3A""",5.6235,4.0415,0.0,3.7481,1.5316,4.831,1.2338,0.0,0.6028,5.6543,0.0,2.3709,0.0,0.0,12.4971,0.0,0.0,12.0808,0.5474,10.22,0.0,0.0,5.2076,0.0,4.3765,5.2724,0.0,0.0,1.686,4.7264,5.3924,13.0392,0.4116,3.7606,7.2546,0.0,…,3.7504,0.0,1.3719,10.6166,0.8555,11.2396,11.1355,12.1301,0.0,10.5382,0.0,1.4781,0.4004,1.3785,2.2788,9.4778,0.8836,15.5583,0.0,6.9493,0.0,13.9854,1.1865,14.4175,0.0,11.8291,1.6428,1.1045,0.3936,2.6442,0.0,8.1001,11.5015,14.25,0.0,0.0,0.5881
"""FABP7""",9.6318,4.5672,4.9863,3.7869,2.6685,1.178,5.7254,0.4815,7.9864,11.4959,0.7861,9.7849,10.1427,0.8598,0.0,2.3118,3.9118,4.3149,4.5895,5.7889,1.051,0.0,1.6277,0.4665,5.8585,1.2851,10.7898,11.2974,7.9493,4.2025,2.3951,0.0,1.7276,4.1585,0.0,3.7394,…,0.0,9.4015,0.0,3.4417,5.0073,0.0,0.0,1.3168,1.09,0.0,8.3559,6.7183,0.7135,3.2928,4.0671,7.0941,2.7898,0.0,4.4278,4.5174,0.0,0.0,0.3994,0.6949,3.6482,4.3138,0.9587,0.6554,1.6762,2.4597,1.08,5.1418,2.5904,5.8941,10.0022,0.0,4.5789
"""DSG1""",3.3121,5.4522,4.2247,2.2197,3.2493,2.4381,0.3453,2.1909,4.8584,5.0107,3.4303,12.757,2.0417,0.493,0.0,0.0,1.7407,0.8002,4.176,7.0039,1.051,0.0,3.7141,1.7185,3.8258,0.0,0.0,1.2706,1.686,0.4032,1.0461,0.7742,3.4029,4.516,0.0,1.4896,…,0.5648,8.3924,8.2062,2.9498,7.3831,1.9147,0.0,0.0,0.3586,0.0,8.4437,3.9001,1.3783,2.433,0.0,4.9711,1.8218,0.0,0.8518,3.1898,0.0,0.0,3.1659,1.7088,2.468,3.9801,3.0136,0.3647,0.0,4.5081,1.3134,6.4787,3.4654,4.908,8.1218,0.8444,6.2333
"""GRIA2""",6.0696,1.5726,2.5984,0.4497,4.0937,1.9414,10.284,4.7628,2.6292,1.109,7.7487,6.6353,3.4425,0.0,0.0,0.0,1.2254,0.8002,0.0,2.5089,3.7555,8.6277,5.287,7.3066,0.3635,1.4459,0.8669,0.0,0.4538,0.4032,1.8728,0.4384,4.5519,1.0538,0.8711,6.5748,…,1.9542,1.3207,0.613,3.7609,1.7775,0.4811,1.0788,3.3142,0.8848,8.5144,8.1203,0.4406,1.1888,0.848,0.4333,4.0163,2.131,9.8821,0.0,8.6982,1.0427,7.3546,9.1116,6.0303,1.161,10.3031,9.6381,2.9821,0.3936,3.615,0.0,4.868,10.1273,2.8022,5.7456,0.8444,0.3238
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

In [35]:
y = annotations.select(
    pl.col("sampleID").alias("sample_ids"),
    pl.col("classes").alias("class"),
)
output_dir = "tcga_brca_data/splits"
OmicDataSplitter(
    df=mrna,
    annotation_cols=['gene'],
    y_df=y,
    n_features=2000,
    n_splits=5,
    random_state=3,
    output_dir=output_dir + "/mrna_genes",
).process_data()

Only 481 samples out of 481 found in y_df


Processing folds: 100%|██████████| 5/5 [1:18:42<00:00, 944.43s/fold]


In [60]:
# based on what genes are selected for mrna, select the same genes for meth and cnv
# perform selection for mirnas
mirna = pl.read_csv(
    "tcga_brca_data/raw/mirna.csv"
)
mirna = mirna.select(['miRNA_ID'] + nonnormal_samples)
mirna

miRNA_ID,TCGA-A1-A0SD-01,TCGA-A1-A0SE-01,TCGA-A1-A0SH-01,TCGA-A1-A0SJ-01,TCGA-A1-A0SK-01,TCGA-A1-A0SM-01,TCGA-A1-A0SO-01,TCGA-A2-A04N-01,TCGA-A2-A04P-01,TCGA-A2-A04Q-01,TCGA-A2-A04R-01,TCGA-A2-A04T-01,TCGA-A2-A04U-01,TCGA-A2-A04V-01,TCGA-A2-A04W-01,TCGA-A2-A04X-01,TCGA-A2-A04Y-01,TCGA-A2-A0CL-01,TCGA-A2-A0CM-01,TCGA-A2-A0CP-01,TCGA-A2-A0CQ-01,TCGA-A2-A0CS-01,TCGA-A2-A0CT-01,TCGA-A2-A0CU-01,TCGA-A2-A0CV-01,TCGA-A2-A0CW-01,TCGA-A2-A0D0-01,TCGA-A2-A0D1-01,TCGA-A2-A0D2-01,TCGA-A2-A0D3-01,TCGA-A2-A0D4-01,TCGA-A2-A0EM-01,TCGA-A2-A0EN-01,TCGA-A2-A0EO-01,TCGA-A2-A0EQ-01,TCGA-A2-A0ER-01,…,TCGA-E2-A14W-01,TCGA-E2-A14X-01,TCGA-E2-A14Y-01,TCGA-E2-A14Z-01,TCGA-E2-A150-01,TCGA-E2-A152-01,TCGA-E2-A153-01,TCGA-E2-A154-01,TCGA-E2-A155-01,TCGA-E2-A156-01,TCGA-E2-A158-01,TCGA-E2-A159-01,TCGA-E2-A15A-01,TCGA-E2-A15C-01,TCGA-E2-A15D-01,TCGA-E2-A15E-01,TCGA-E2-A15F-01,TCGA-E2-A15G-01,TCGA-E2-A15H-01,TCGA-E2-A15I-01,TCGA-E2-A15J-01,TCGA-E2-A15K-01,TCGA-E2-A15L-01,TCGA-E2-A15M-01,TCGA-E2-A15O-01,TCGA-E2-A15P-01,TCGA-E2-A15R-01,TCGA-E2-A15S-01,TCGA-E2-A15T-01,TCGA-E2-A1AZ-01,TCGA-E2-A1B0-01,TCGA-E2-A1B1-01,TCGA-E2-A1B4-01,TCGA-E2-A1B5-01,TCGA-E2-A1B6-01,TCGA-E2-A1BC-01,TCGA-E2-A1BD-01
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""hsa-let-7a-1""",14.03039,13.034042,12.962609,13.091325,12.207688,13.126299,11.735671,11.72943,13.141287,12.213246,12.856138,13.33289,13.249776,13.107813,12.327013,12.668052,13.00873,13.023521,12.538091,12.974924,13.591946,13.682025,12.963904,12.982559,13.780127,13.002531,12.959375,12.290943,12.548272,14.355641,13.386397,13.517807,12.563069,13.073552,13.055749,14.097343,…,12.933047,13.412834,13.295752,13.380763,12.924364,13.338225,13.589713,13.01339,13.934445,14.307726,13.260388,11.847683,13.721022,13.791229,13.285355,13.630036,13.679882,13.258044,12.769781,14.056884,14.205192,13.438621,13.580108,13.102478,13.71765,13.701193,13.498005,14.709119,14.608977,12.666225,13.206409,13.563254,15.086711,13.662556,13.701712,13.904604,14.026616
"""hsa-let-7a-2""",14.018248,13.017645,12.971607,13.071205,12.2287,13.131248,11.72821,11.711106,13.121908,12.222766,12.859981,13.320862,13.247731,13.109839,12.327817,12.6326,13.010961,12.999008,12.517011,13.013343,13.569417,13.681565,12.969242,12.947125,13.785213,13.000689,12.943465,12.299641,12.553494,14.356464,13.367273,13.510865,12.550192,13.071606,13.02276,14.067662,…,12.921891,13.409076,13.305842,13.371711,12.934441,13.332303,13.57188,13.028045,13.934515,14.304563,13.267611,11.847515,13.697301,13.778067,13.2757,13.633115,13.661747,13.222523,12.760095,14.052668,14.199054,13.437686,13.566287,13.095546,13.7218,13.701746,13.484103,14.702159,14.60681,12.680116,13.205629,13.548641,15.097745,13.660415,13.709611,13.897027,14.033012
"""hsa-let-7a-3""",14.024003,13.036697,12.983236,13.101082,12.232114,13.176328,11.769435,11.75483,13.1455,12.273188,12.846426,13.383369,13.245801,13.136332,12.357711,12.630557,13.020616,13.045014,12.534349,13.009295,13.631521,13.677169,12.984002,12.992206,13.784367,13.006994,12.954434,12.33224,12.553059,14.362705,13.396629,13.519213,12.582631,13.101982,13.083498,14.098052,…,12.956874,13.417069,13.321482,13.410418,12.94685,13.340087,13.593498,13.023757,13.950362,14.317231,13.268565,11.921041,13.745192,13.793379,13.304198,13.650106,13.685244,13.219345,12.758886,14.066863,14.209779,13.466337,13.58083,13.100015,13.726457,13.70946,13.506195,14.716166,14.622777,12.703487,13.214104,13.572047,15.093364,13.645393,13.720637,13.917199,14.030702
"""hsa-let-7b""",15.632852,15.423974,15.481879,15.589285,14.494925,16.019204,12.786965,13.28372,14.062908,13.713737,14.890818,14.077445,13.991182,14.544838,13.679741,14.591142,15.459013,13.86464,13.418417,14.286339,16.013266,15.444711,14.981333,14.237693,15.258472,13.144742,15.787212,13.711375,

In [51]:
y

sample_ids,class
str,i64
"""TCGA-A1-A0SD-01""",2
"""TCGA-A1-A0SE-01""",2
"""TCGA-A1-A0SH-01""",2
"""TCGA-A1-A0SJ-01""",2
"""TCGA-A1-A0SK-01""",0
…,…
"""TCGA-E2-A1B4-01""",2
"""TCGA-E2-A1B5-01""",0
"""TCGA-E2-A1B6-01""",2


In [63]:
mirna = preprocess_seq_data(
    df = mirna,
    annotation_cols=['miRNA_ID'],
    min_median_expression=2,
    min_detection_rate=0.4,
    top_n=10000,
    cpm_normalization=False,
    log_transform=False,
)
mirna
OmicDataSplitter(
    df=mirna,
    annotation_cols=['miRNA_ID'],
    y_df=y,
    n_features=-1,
    n_splits=5,
    random_state=3,
    output_dir=output_dir + "/mirna",
).process_data()

Only 481 samples out of 481 found in y_df


0it [00:00, ?it/s]  0%|          | 0/5 [00:00<?, ?fold/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s] 40%|████      | 2/5 [00:00<00:00, 15.41fold/s]
0it [00:00, ?it/s]
Processing folds: 100%|██████████| 5/5 [00:00<00:00, 14.83fold/s]


In [75]:
mrna

gene,TCGA-A1-A0SD-01,TCGA-A1-A0SE-01,TCGA-A1-A0SH-01,TCGA-A1-A0SJ-01,TCGA-A1-A0SK-01,TCGA-A1-A0SM-01,TCGA-A1-A0SO-01,TCGA-A2-A04N-01,TCGA-A2-A04P-01,TCGA-A2-A04Q-01,TCGA-A2-A04R-01,TCGA-A2-A04T-01,TCGA-A2-A04U-01,TCGA-A2-A04V-01,TCGA-A2-A04W-01,TCGA-A2-A04X-01,TCGA-A2-A04Y-01,TCGA-A2-A0CL-01,TCGA-A2-A0CM-01,TCGA-A2-A0CP-01,TCGA-A2-A0CQ-01,TCGA-A2-A0CS-01,TCGA-A2-A0CT-01,TCGA-A2-A0CU-01,TCGA-A2-A0CV-01,TCGA-A2-A0CW-01,TCGA-A2-A0D0-01,TCGA-A2-A0D1-01,TCGA-A2-A0D2-01,TCGA-A2-A0D3-01,TCGA-A2-A0D4-01,TCGA-A2-A0EM-01,TCGA-A2-A0EN-01,TCGA-A2-A0EO-01,TCGA-A2-A0EQ-01,TCGA-A2-A0ER-01,…,TCGA-E2-A14W-01,TCGA-E2-A14X-01,TCGA-E2-A14Y-01,TCGA-E2-A14Z-01,TCGA-E2-A150-01,TCGA-E2-A152-01,TCGA-E2-A153-01,TCGA-E2-A154-01,TCGA-E2-A155-01,TCGA-E2-A156-01,TCGA-E2-A158-01,TCGA-E2-A159-01,TCGA-E2-A15A-01,TCGA-E2-A15C-01,TCGA-E2-A15D-01,TCGA-E2-A15E-01,TCGA-E2-A15F-01,TCGA-E2-A15G-01,TCGA-E2-A15H-01,TCGA-E2-A15I-01,TCGA-E2-A15J-01,TCGA-E2-A15K-01,TCGA-E2-A15L-01,TCGA-E2-A15M-01,TCGA-E2-A15O-01,TCGA-E2-A15P-01,TCGA-E2-A15R-01,TCGA-E2-A15S-01,TCGA-E2-A15T-01,TCGA-E2-A1AZ-01,TCGA-E2-A1B0-01,TCGA-E2-A1B1-01,TCGA-E2-A1B4-01,TCGA-E2-A1B5-01,TCGA-E2-A1B6-01,TCGA-E2-A1BC-01,TCGA-E2-A1BD-01
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""CYP2A6""",0.7326,0.9907,0.0,7.0156,0.3953,0.0,0.0,0.4815,0.6028,0.4706,5.0547,0.4305,1.3554,2.6545,4.0591,0.7832,0.4164,2.9395,0.5474,1.2268,3.7932,8.0671,2.4851,2.8297,1.7191,2.8724,0.0,0.0,1.0765,0.0,1.0461,13.0051,3.158,1.6563,0.0,4.1354,…,1.5443,1.1688,0.0,11.6415,0.8555,0.0,0.0,3.0799,1.09,0.0,0.0,0.0,4.0318,1.1375,2.3794,3.771,8.3743,8.0539,11.3892,4.3017,4.3529,4.1877,0.0,14.5701,2.3563,8.4825,1.6428,2.9821,3.0383,0.0,1.5143,2.9513,0.6123,15.9084,0.0,2.3155,1.5912
"""CLEC3A""",5.6235,4.0415,0.0,3.7481,1.5316,4.831,1.2338,0.0,0.6028,5.6543,0.0,2.3709,0.0,0.0,12.4971,0.0,0.0,12.0808,0.5474,10.22,0.0,0.0,5.2076,0.0,4.3765,5.2724,0.0,0.0,1.686,4.7264,5.3924,13.0392,0.4116,3.7606,7.2546,0.0,…,3.7504,0.0,1.3719,10.6166,0.8555,11.2396,11.1355,12.1301,0.0,10.5382,0.0,1.4781,0.4004,1.3785,2.2788,9.4778,0.8836,15.5583,0.0,6.9493,0.0,13.9854,1.1865,14.4175,0.0,11.8291,1.6428,1.1045,0.3936,2.6442,0.0,8.1001,11.5015,14.25,0.0,0.0,0.5881
"""FABP7""",9.6318,4.5672,4.9863,3.7869,2.6685,1.178,5.7254,0.4815,7.9864,11.4959,0.7861,9.7849,10.1427,0.8598,0.0,2.3118,3.9118,4.3149,4.5895,5.7889,1.051,0.0,1.6277,0.4665,5.8585,1.2851,10.7898,11.2974,7.9493,4.2025,2.3951,0.0,1.7276,4.1585,0.0,3.7394,…,0.0,9.4015,0.0,3.4417,5.0073,0.0,0.0,1.3168,1.09,0.0,8.3559,6.7183,0.7135,3.2928,4.0671,7.0941,2.7898,0.0,4.4278,4.5174,0.0,0.0,0.3994,0.6949,3.6482,4.3138,0.9587,0.6554,1.6762,2.4597,1.08,5.1418,2.5904,5.8941,10.0022,0.0,4.5789
"""DSG1""",3.3121,5.4522,4.2247,2.2197,3.2493,2.4381,0.3453,2.1909,4.8584,5.0107,3.4303,12.757,2.0417,0.493,0.0,0.0,1.7407,0.8002,4.176,7.0039,1.051,0.0,3.7141,1.7185,3.8258,0.0,0.0,1.2706,1.686,0.4032,1.0461,0.7742,3.4029,4.516,0.0,1.4896,…,0.5648,8.3924,8.2062,2.9498,7.3831,1.9147,0.0,0.0,0.3586,0.0,8.4437,3.9001,1.3783,2.433,0.0,4.9711,1.8218,0.0,0.8518,3.1898,0.0,0.0,3.1659,1.7088,2.468,3.9801,3.0136,0.3647,0.0,4.5081,1.3134,6.4787,3.4654,4.908,8.1218,0.8444,6.2333
"""GRIA2""",6.0696,1.5726,2.5984,0.4497,4.0937,1.9414,10.284,4.7628,2.6292,1.109,7.7487,6.6353,3.4425,0.0,0.0,0.0,1.2254,0.8002,0.0,2.5089,3.7555,8.6277,5.287,7.3066,0.3635,1.4459,0.8669,0.0,0.4538,0.4032,1.8728,0.4384,4.5519,1.0538,0.8711,6.5748,…,1.9542,1.3207,0.613,3.7609,1.7775,0.4811,1.0788,3.3142,0.8848,8.5144,8.1203,0.4406,1.1888,0.848,0.4333,4.0163,2.131,9.8821,0.0,8.6982,1.0427,7.3546,9.1116,6.0303,1.161,10.3031,9.6381,2.9821,0.3936,3.615,0.0,4.868,10.1273,2.8022,5.7456,0.8444,0.3238
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

In [26]:
# load each mrna train and test split and select the same features and samples for meth and cnv
meth = pl.read_csv("tcga_brca_data/raw/meth.csv")
cnv = pl.read_csv("tcga_brca_data/raw/cnv.csv")

mrna_path = "tcga_brca_data/splits/mrna"
meth_path = "tcga_brca_data/splits/meth"
cnv_path = "tcga_brca_data/splits/cnv"
mrna_train = mrna_path + "/train"
mrna_test = mrna_path + "/test"

mrna_train_files = os.listdir(mrna_train)
mrna_test_files = os.listdir(mrna_test)
mrna_train_files.sort()
mrna_test_files.sort()

def format_by_df(df, trn_df, tst_df, gene_col='gene'):

    selected_features = mrna_trn_df.columns[:-2]
    selected_features_tst = mrna_tst_df.columns[:-2]
    assert selected_features == selected_features_tst

    train_samples, train_classes = trn_df['sample_ids'].to_list(), trn_df['class'].to_list()
    test_samples, test_classes = tst_df['sample_ids'].to_list(), tst_df['class'].to_list()
    
    df_train = df.select([gene_col] + train_samples)
    df_test = df.select([gene_col] + test_samples)

    # Get genes as column names
    genes_trn = df_train['gene'].to_list()
    genes_tst = df_test['gene'].to_list()

    # Drop gene column and convert to numpy for transpose
    values_trn = df_train.drop('gene').to_numpy().T
    values_tst = df_test.drop('gene').to_numpy().T

    # Create new dataframe with genes as columns
    df_train = pl.DataFrame(
        values_trn,
        schema=genes_trn,
    )
    df_test = pl.DataFrame(
        values_tst,
        schema=genes_tst,
    )

    # Select features and add sample_ids and class columns
    df_train = df_train.select(selected_features).with_columns([
        pl.Series(name='sample_ids', values=train_samples),
        pl.Series(name='class', values=train_classes)
    ])
    df_test = df_test.select(selected_features).with_columns([
        pl.Series(name='sample_ids', values=test_samples),
        pl.Series(name='class', values=test_classes)
    ])

    return df_train, df_test


for i, (tr_f, ts_f) in enumerate(zip(mrna_train_files, mrna_test_files)):
    mrna_trn_df = pl.read_csv(f"{mrna_train}/{tr_f}")
    mrna_tst_df = pl.read_csv(f"{mrna_test}/{ts_f}")

    meth_train_split, meth_test_split = format_by_df(meth, mrna_trn_df, mrna_tst_df)
    cnv_train_split, cnv_test_split = format_by_df(meth, mrna_trn_df, mrna_tst_df)

    # write to split files
    meth_train_split.write_csv(f"{meth_path}/train/{tr_f}")
    meth_test_split.write_csv(f"{meth_path}/test/{ts_f}")
    cnv_train_split.write_csv(f"{cnv_path}/train/{tr_f}")
    cnv_test_split.write_csv(f"{cnv_path}/test/{ts_f}")
